### Подготовка датасета по показателю объем убоя овец и коз

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

from pylab import rcParams
from IPython.display import display
import math
from prophet import Prophet
pd.set_option('display.max_columns', 130)


import warnings
from statsmodels.tools.sm_exceptions import InterpolationWarning
warnings.simplefilter("ignore", category=InterpolationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)



In [2]:
df = pd.read_excel("../../Data cleansing/output data/Просуммированные по категориям с доп регрессорами.xlsx")
df.head(5)

,Показатель,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,Верблюды,АКМОЛИНСКАЯ ОБЛАСТЬ,0.00,0.00,0.40,0.00,0.00,0.00,0.09,1.00,0.00,0.00,0.20,0.00,0.00,0.18,0.28,0.00,0.00,0.40,0.00,0.00,0.65,0.00,0.31,1.00,0.00,0.00,0.00,0.00,0.00,2.01,0.00,1.20,0.00,0.00,2.18,0.39,0.00,0.00,1.04,0.00,0.14,2.08,0.00,0.00,0.00,0.00,0.00,0.30,0.00,0.00,0.00,0.66,0.00,0.33,0.00,0.9,0.00,0.00,0.00,10.08,0.00,0.0,0.00,0.0,0.00,0.54,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.36,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.40,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.00
1,Верблюды,АКТЮБИНСКАЯ ОБЛАСТЬ,101.98,67.47,374.84,115.59,218.72,14.15,19.77,3.00,39.16,46.16,238.56,463.35,109.04,72.94,384.96,114.35,221.89,10.93,18.79,3.50,38.53,46.64,216.08,472.07,110.32,68.17,380.15,127.12,217.61,14.98,21.26,7.14,44.63,51.78,239.43,513.33,115.70,69.40,371.53,130.17,218.41,14.59,21.76,7.19,45.97,55.48,252.68,527.66,117.94,70.37,385.75,117.80,218.45,15.54,21.78,7.2,47.06,58.02,257.54,543.35,119.80,71.3,396.90,121.8,228.10,15.90,21.9,7.2,63.00,59.90,260.20,552.10,121.60,71.8,398.40,128.50,228.80,13.10,22.50,7.20,63.90,61.90,150.70,554.90,121.30,73.3,404.20,128.70,234.90,13.60,23.60,6.3,49.80,62.4,153.70,552.8,118.07,69.17,384.21,121.01,223.46,11.76,21.39,5.52,48.18,61.39,150.87,531.79,119.50,72.60,391.0,126.1,229.10,12.2,24.10,5.7,49.87,63.00,154.3,538.40
2,Верблюды,АЛМАТИНСКАЯ ОБЛАСТЬ,1.00,0.20,51.60,25.40,0.00,61.90,76.87,95.57,16.09,0.10,10.80,46.78,15.06,13.70,131.80,9.20,17.89,130.12,2.00,0.00,4.99,2.06,4.90,44.48,15.06,23.94,41.60,19.54,3.00,15.14,18.50,14.82,14.52,12.60,9.92,43.11,20.72,2.96,15.24,0.00,1.00,20.03,3.44,7.77,117.25,1.00,0.00,20.34,21.95,3.04,17.91,26.81,9.16,18.80,13.37,0.0,17.29,15.90,4.69,23.10,10.85,9.7,42.75,0.0,6.21,21.10,4.0,2.6,28.10,4.53,79.50,93.18,23.70,11.2,16.00,2.00,3.10,19.62,2.30,4.85,11.18,2.80,9.65,36.44,10.60,11.5,26.10,7.20,12.75,11.20,11.20,27.0,25.26,17.9,21.90,12.5,16.55,16.81,22.78,12.10,12.44,6.85,39.65,17.22,8.85,27.48,0.48,29.72,18.90,17.40,12.5,16.4,11.60,16.7,15.70,24.0,7.70,6.90,5.4,12.60
3,Верблюды,АТЫРАУСКАЯ ОБЛАСТЬ,213.89,167.70,306.60,164.97,342.57,192.00,43.60,113.03,262.37,193.97,308.17,1087.33,325.10,190.60,301.10,154.84,328.50,220.30,66.10,118.20,234.90,196.51,270.92,974.18,303.30,167.41,323.54,182.34,349.10,249.40,57.57,88.40,221.21,199.63,278.80,964.02,305.22,159.62,326.87,159.50,367.20,257.50,60.01,78.10,253.27,332.00,355.05,989.86,280.08,165.05,334.51,134.85,367.97,321.22,126.22,93.7,263.10,366.66,348.54,1041.38,293.40,154.8,339.44,143.6,405.80,308.35,88.5,130.5,575.30,458.70,357.62,998.10,292.76,188.3,487.02,129.28,421.44,351.20,95.77,97.54,624.80,631.65,499.30,1179.66,286.67,189.8,513.16,171.46,402.10,432.22,121.53,126.2,637.50,651.7,482.94,1412.2,307.85,200.59,466.60,182.63,414.56,405.19,184.80,154.96,717.92,719.01,522.83,1164.77,323.81,423.76,441.9,219.1,487.69,446.9,191.43,133.5,730.34,709.68,537.6,930.28
4,Верблюды,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,0.30,0.00,0.00,1.14,6.12,4.70

In [3]:
df['Показатель'].unique()

array(['Верблюды', 'КРС', 'Лошади', 'Молоко', 'Овцы и козы', 'Птица',
       'Свиньи', 'Яйца', 'Температура', 'Поголовье: КРС',
       'Поголовье: лошади', 'Поголовье: овцы и козы', 'Поголовье: свиньи',
       'Поголовье: птица домашняя', 'Поголовье: верблюды', 'Осадки',
       'Цена: Говядина', 'Цена: Баранина', 'Цена: Молоко', 'Цена: Яйца'],
      dtype=object)

In [4]:
df_sheep = df[df['Показатель'].isin(['Овцы и козы', 'Температура', 'Осадки', 'Поголовье: овцы и козы', 'Цена: Баранина'])]
df_sheep.sample(10)

,Показатель,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
245,Поголовье: овцы и козы,ГШЫМКЕНТ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,118134.000000,115213.000000,112087.000000,109172.000000,106248.000000,102310.000000,113182.000000,110026.000000,107594.000000,105673.000000,105992.000000,108903.000000,106287.000000,118033.000000,115094.000000,112562.000000,109741.000000,106307.000000,95382.000000,101122.000000,110105.000000,107462.000000,105400.000000,105432.000000,108511.000000,108640.000000,105668.000000,103182.000000,101006.000000,98734.000000,96255.000000,91469.000000,88635.000000,86608.000000,87580.000000,88939.000000,93362.000000,93826.000000,92681.000000,91298.000000,89714.000000,88030.000000,85231.000000,74432.000000,71333.000000,68902.000000,68350.000000,69114.000000,71809.000000,77231.000000,75862.000000,74497.000000,89166.000000,87615.000000,86078.000000,6952.000000,4582.000000,4581.000000,4581.000000,4789.000000,4758.000000,2994.000000,4746.000000,4229.000000,4259.000000,4677.000000,5169.000000,113201.000000,109397.000000,106922.000000,105050.000000,104750.000000,102473.000000,144113.000000,142539.000000,139792.000000,137703.000000,137234.000000,135695.000000
183,Температура,ГШЫМКЕНТ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.143333,29.074194,25.683871,20.013333,12.270968,3.753333,1.993548,2.974194,3.014286,10.848387,13.730000,19.896774,23.856667,29.738710,26.780645,19.666667,14.077419,3.173333,3.067742,-1.006452,4.727586,10.148387,14.743333,20.593548,24.523333,27.819355,25.422581,18.966667,11.438710,2.146667,-2.748387,-0.154839,5.196429,7.132258,14.433333,22.338710,26.903333,29.264516,27.780645,21.900000,10.503226,4.300000,5.248387,2.851613,3.875000,6.661290,18.320000,20.200000,26.040000,28.654839,25.396774,23.286667,13.406452,6.720000,-1.938710,-5.032258,3.796429,13.312903,15.433333,20.448387,27.523333,29.961290,25.861290,19.526667,15.087097,10.796667,2.519355,3.425806,2.065517,7.506452,15.110000,18.980645,26.646667,27.029032,26.658065,19.833333,13.122581,7.926667,-1.200000
368,Цена: Баранина,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,100.000000,100.000000,100.000000,100.800000,101.203200,101.203200,101.203200,101.203200,101.203200,101.203200,101.203200,101.203200,101.203200,101.203200,101.203200,103.632077,104.253869,105.400662,104.557457,104.557457,104.557457,104.557457,104.557457,104.243784,105.077734,110.646854,110.646854,110.646854,110.646854,110.757501,110.757501,110.757501,115.298559,115.298559,115.298559,115.413857,115.413857,115.413857,118.645445,118.645445,118.645445,118.645445,118.645445,118.645445,118.645445,118.645445,118.645445,120.543772,122.954648,122.954648,122.954648,123.938285,124.681915,128.297690,131.120240,132.562562,137.865065,140.484501,140.484501,140.484501,140.484501,143.294191,150.172312,150.773001,152.129958,153.042738,154.726208,157.511280,160.976528,

In [5]:
# Step 1: Pivot to wide format (each indicator becomes columns of periods)
df_wide = df_sheep.pivot(index="Регион", columns="Показатель")

# Step 2: Flatten multi-level columns: ('2015-01', 'КРС') → 'КРС_2015-01'
df_wide.columns = [f"{col[1]}_{col[0]}" for col in df_wide.columns]
df_wide = df_wide.reset_index()

# Step 3: Melt: one row per region-period-indicator
df_melted = df_wide.melt(id_vars="Регион", var_name="indicator_period", value_name="value")

# Step 4: Extract 'Период' and 'Показатель' from the combined column
df_melted["Период"] = df_melted["indicator_period"].str.extract(r"_(\d{4}-\d{2})$")
df_melted["Показатель"] = df_melted["indicator_period"].str.extract(r"^(.+)_\d{4}-\d{2}")

# Step 5: Pivot again to get final modeling format: one row per region+period, one column per indicator
df_sheep = df_melted.pivot_table(index=["Регион", "Период"], columns="Показатель", values="value").reset_index()
print(df_sheep.groupby("Регион").size().reset_index(name="Количество строк"))
df_sheep

                            Регион  Количество строк
0              АКМОЛИНСКАЯ ОБЛАСТЬ               120
1              АКТЮБИНСКАЯ ОБЛАСТЬ               120
2              АЛМАТИНСКАЯ ОБЛАСТЬ               120
3               АТЫРАУСКАЯ ОБЛАСТЬ               120
4   ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ               120
5                          ГАЛМАТЫ               120
6                          ГАСТАНА               120
7                         ГШЫМКЕНТ                79
8               ЖАМБЫЛСКАЯ ОБЛАСТЬ               120
9    ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ               120
10          КАРАГАНДИНСКАЯ ОБЛАСТЬ               120
11            КОСТАНАЙСКАЯ ОБЛАСТЬ               120
12          КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ               120
13           МАНГИСТАУСКАЯ ОБЛАСТЬ               120
14                    ОБЛАСТЬ АБАЙ                31
15                  ОБЛАСТЬ ЖЕТІСУ                31
16                  ОБЛАСТЬ ҰЛЫТАУ                31
17            ПАВЛОДАРСКАЯ ОБЛАСТЬ            

Показатель,Регион,Период,Овцы и козы,Осадки,Поголовье: овцы и козы,Температура,Цена: Баранина
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-01,602.28,9.8,470582.0,-12.490323,100.000000
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-02,636.18,9.8,524810.0,-10.192857,100.000000
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-03,740.73,8.3,562860.0,-5.870968,100.000000
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-04,690.81,8.8,599518.0,4.490000,99.200000
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-05,869.73,42.8,632220.0,14.574194,99.200000
...,...,...,...,...,...,...,...
2166,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,3378.76,0.0,5034130.0,27.874194,179.022742
2167,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,4625.15,0.5,4967650.0,20.766667,179.201765
2168,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-10,4443.92,13.6,4881045.0,13.200000,179.380967
2169,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-11,5402.06,12.1,4779726.0,6.500000,179.380967


In [6]:
df_sheep = df_sheep[df_sheep["Регион"] != 'РЕСПУБЛИКА КАЗАХСТАН']
df_sheep = df_sheep.drop(columns=['Осадки', 'Поголовье: овцы и козы', 'Температура',
       'Цена: Баранина'])
df_sheep.sample(10)

Показатель,Регион,Период,Овцы и козы
1660,ОБЛАСТЬ АБАЙ,2024-03,1295.56
730,ГАСТАНА,2015-11,7.03
480,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,2015-01,841.83
1544,МАНГИСТАУСКАЯ ОБЛАСТЬ,2017-02,238.23
1683,ОБЛАСТЬ ЖЕТІСУ,2023-07,694.26
1635,МАНГИСТАУСКАЯ ОБЛАСТЬ,2024-09,452.73
277,АЛМАТИНСКАЯ ОБЛАСТЬ,2018-02,1332.89
682,ГАЛМАТЫ,2021-11,0.40
1295,КОСТАНАЙСКАЯ ОБЛАСТЬ,2016-05,124.32
2113,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2020-03,5949.45


In [7]:
df_sheep.to_excel("Датасет по овцы и козы.xlsx", index=False)